In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')

def ShiftAndSum(signal, x_range, y_range, freq):
    
    nch = signal.shape[0]
    angles = np.radians(np.arange(0, nch, 1) * 360 / nch)
    r_array = 0.10
    wavelength_lo = 3e8 / 25.86e9
    Nsample = 8192
    fsample = 200e6
    
    grad_b_angles = 2 * np.pi * np.arange(0, Nsample, 1) * freq / fsample
    
    x_antenna = r_array * np.cos(angles)
    y_antenna = r_array * np.sin(angles)
    
    #x_antenna = x_antenna.reshape((x_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    #y_antenna = y_antenna.reshape((y_antenna.size, 1)).repeat(signal.shape[-1], axis = -1)
    
    x_grid, y_grid = np.meshgrid(x_range, y_range)
    
    x_grid = x_grid.flatten()
    y_grid = y_grid.flatten()
    
    r_grid = np.sqrt(x_grid ** 2 + y_grid ** 2)
    
    theta_grid = np.arctan2(-y_grid, x_grid)
    theta_grid_grad_b = theta_grid.reshape(theta_grid.size, 1) + grad_b_angles
    #theta_grid_grad_b = theta_grid.reshape((x_range.size, y_range.size, 1)).repeat(Nsample, axis=-1) + grad_b_phases
    
    #print(r_grid.shape, theta_grid_grad_b.shape, r_array, angles.shape)
    
    #delta_theta = angles.reshape((angles.size, 1, 1)) - theta_grid_grad_b.reshape((1, *theta_grid_grad_b.shape)) 
    
    x_grad_b = r_grid.reshape((r_grid.size, 1)) * np.cos(theta_grid_grad_b)
    y_grad_b = r_grid.reshape((r_grid.size, 1)) * np.sin(theta_grid_grad_b)
    
    #print(x_grad_b.shape, y_grad_b.shape)
    
    d_grad_b = np.sqrt((x_antenna.reshape((x_antenna.size, 1, 1)) - x_grad_b.reshape((1, *x_grad_b.shape))) ** 2 + (y_antenna.reshape((y_antenna.size, 1, 1)) - y_grad_b.reshape((1, *y_grad_b.shape))) ** 2)
    
    #source_angle = -1 * 2 * np.pi * np.arange(0, Nsample, 1) * freq / fsample
    
    #r_new = np.sqrt(x_new ** 2 + y_new ** 2)
    
    #x_new = r_new * np.cos(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    #y_new = r_new * np.sin(source_angle).reshape((1, Nsample)).repeat(nch, axis=0)
    
    #d_old = np.sqrt(x_antenna ** 2 + y_antenna ** 2)
    #d_new = np.sqrt((x_new - x_antenna) ** 2 + (y_new - y_antenna) ** 2)
    
    phase_shift = 2 * np.pi * (d_grad_b) / wavelength_lo + angles.reshape((angles.size, 1, 1))
    #print(phase_shift.shape, signal.shape)
    
    shifted_signal = np.exp(-1j * phase_shift) * signal.reshape((signal.shape[0], 1, signal.shape[-1]))
    
    return shifted_signal.sum(axis=0)
    
#sum_signal = ShiftAndSum(np.zeros((60, 8192), np.complex64), np.linspace(-0.03, 0.03, 51), np.linspace(-0.03, 0.03, 51), 1e3)



# load data

In [ ]:
os.listdir(os.path.join(DATAPATH))

In [ ]:
os.listdir(os.path.join(RESULTPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '210930_87_18595_energy_angle_test_slice8192.h5'))
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211027_84_25_2cm.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

# compute scores

# load meta data

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)
#test_metadata = pd.DataFrame(test_data.metadata)

In [ ]:
energy_array = template_metadata['energy'].array
angle_array = template_metadata['theta_min'].array

#test_energies = np.array(test_metadata['energy'].array).squeeze()
#test_angles = np.array(test_metadata['theta_min'].array).squeeze()

unique_energy = np.sort(np.unique(template_metadata['energy'].array))
unique_angle = np.sort(np.unique(template_metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)

sorting_indexes = []

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    sorting_indexes.append(template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0])
    
sorting_indexes = np.array(sorting_indexes)

print(sorting_indexes.shape)

In [ ]:
print(unique_energy, unique_angle)

# grad-b shift beamforming, signal 1

In [ ]:
signal_energy = 18595
signal_angle = 88.0

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 3 * 8192))[:, 0:8192]
print(signal.shape)

ngrid = 31
x_pos_range = np.linspace(-0.03, 0.03, ngrid)
y_pos_range = np.linspace(-0.03, 0.03, ngrid)
grad_b_freq = 0

#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

sum_signal = ShiftAndSum(signal, x_pos_range, y_pos_range, grad_b_freq)

In [ ]:
sns.set_theme(style='ticks', context='paper')
cmap = sns.color_palette('mako_r', as_cmap=True)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

image = np.flip((abs(sum_signal) ** 2).sum(axis=-1).reshape((ngrid, ngrid)), axis=1)
print(image.max())

img = ax.imshow(image, cmap = cmap, aspect='auto', interpolation='none')

cbar = fig.colorbar(img)

# grad-b shift beamforming, signal 2

In [ ]:
signal_energy = 18595
signal_angle = 85.

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 3 * 8192))[:, 0:8192]

ngrid = 21
x_pos_range = np.linspace(-0.03, 0.03, ngrid)
y_pos_range = np.linspace(-0.03, 0.03, ngrid)
grad_b_freq = 0.0

#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

sum_signal = ShiftAndSum(signal, x_pos_range, y_pos_range, grad_b_freq)

In [ ]:
sns.set_theme(style='ticks', context='paper')
cmap = sns.color_palette('mako_r', as_cmap=True)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

image = np.flip((abs(sum_signal) ** 2).sum(axis=-1).reshape((ngrid, ngrid)), axis=1,)
print(image.max())

img = ax.imshow(image, cmap = cmap, aspect='auto', interpolation='none',)

cbar = fig.colorbar(img)

# plot power for range of grad-b corrections, signal 1

In [ ]:
signal_energy = 18595.25
signal_angle = 89.6125

grad_b_freq_range = np.linspace(2350, 2450, 61)
power_range = np.zeros(61)

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 8192))

ngrid = 21
x_pos_range = np.linspace(-0.01, -0.03, ngrid)
y_pos_range = np.linspace(-0.01, 0.01, ngrid)

for i, grad_b_freq in enumerate(grad_b_freq_range):


#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

    sum_signal = ShiftAndSum(signal, x_pos_range, y_pos_range, grad_b_freq)
    image = np.flip((abs(sum_signal) ** 2).sum(axis=-1).reshape((ngrid, ngrid)), axis=1)
    
    power_range[i] = image.max()
    print(i+1)

# plot power for range of grad-b corrections, signal 2

In [ ]:
signal_energy = 18596
signal_angle = 89.6

grad_b_freq_range = np.linspace(2350, 2450, 61)
power_range_2 = np.zeros(61)

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 8192))

ngrid = 21
x_pos_range = np.linspace(-0.01, -0.03, ngrid)
y_pos_range = np.linspace(-0.01, 0.01, ngrid)

for i, grad_b_freq in enumerate(grad_b_freq_range):


#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

    sum_signal = ShiftAndSum(signal, x_pos_range, y_pos_range, grad_b_freq)
    image = np.flip((abs(sum_signal) ** 2).sum(axis=-1).reshape((ngrid, ngrid)), axis=1)
    
    power_range_2[i] = image.max()
    print(i+1)

# plot power for range of grad-b corrections, signal 3

In [ ]:
signal_energy = 18596
signal_angle = 89.6

grad_b_freq_range = np.linspace(2e3, 4e3, 31)
power_range_3 = np.zeros(31)

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 8192))

ngrid = 21
x_pos_range = np.linspace(-0.01, -0.03, ngrid)
y_pos_range = np.linspace(-0.01, 0.01, ngrid)

for i, grad_b_freq in enumerate(grad_b_freq_range):


#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

    sum_signal = ShiftAndSum(signal, x_pos_range, y_pos_range, grad_b_freq)
    image = np.flip((abs(sum_signal) ** 2).sum(axis=-1).reshape((ngrid, ngrid)), axis=1)
    
    power_range_3[i] = image.max()
    print(i+1)

In [ ]:
sns.set_theme(style='whitegrid', context='paper')
clist = sns.color_palette('deep')
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(grad_b_freq_range, power_range, '.', color=clist[0])
ax.plot(grad_b_freq_range, power_range_2, '.', color=clist[1])

#ax.set_xlim(2250, 2550)

#ax.set_ylim(2.5e-9, 2.7e-9)
#ax.plot(grad_b_freq_range, power_range_3, '.', color=clist[2])

ax.plot(grad_b_freq_range[power_range.argmax()], power_range.max(), marker='*', markersize=12, color=clist[3])
ax.plot(grad_b_freq_range[power_range_2.argmax()], power_range_2.max(), marker='*', markersize=12, color=clist[3])
#ax.plot(grad_b_freq_range[power_range_3.argmax()], power_range_3.max(), marker='*', markersize=12, color=clist[3])

#ax.vlines(grad_b_freq_range[power_range.argmax()], 1.5e-9, 2.7e-9, color=clist[0])
#ax.vlines(grad_b_freq_range[power_range_2.argmax()], 1.5e-9, 2.7e-9, color=clist[1])